In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from tqdm import tqdm,trange
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# !pip install -q -U trl transformers git+https://github.com/huggingface/peft.git
# !pip install bitsandbytes
# !pip install accelerate

In [ ]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
# import os
# import torch
# from trl import SFTTrainer
# from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig, AutoTokenizer
# from datasets import Dataset
# from transformers import TrainingArguments
# from peft import LoraConfig, get_peft_model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob, pandas as pd
files = glob.glob("/content/drive/MyDrive/dataset.xlsx")

dataset = pd.DataFrame(columns=['Sentence', 'Agent', 'CU', 'Entity', 'Attribute'])

for i in range(len(files)):
    df = pd.read_excel(files[i])
    if all(x in df.columns for x in ['Sentence','Agent','CU','Entity','Attribute']):
      dataset = pd.concat([dataset,df[['Sentence','Agent','CU','Entity','Attribute']]],ignore_index=True)

dataset #display dataframe of combined datas across multiple domain for Sentence, Agent, CU, Attribute columns

,Sentence,Agent,CU,Entity,Attribute
0,Agro Product manager adds new product using A...,Agro Product manager,adds new product,Agro Product,"Product Name, Category ID, Quantity Per Unit,..."
1,And the Logistics coordinator schedules deliv...,the Logistics coordinator,schedules delivery,Order,"Order ID, Delivery Date, Shipping Method, Fre..."
2,And the Warehouse operator prepares shipment ...,the Warehouse operator,prepares shipment,Order,"Order ID, Delivery Address, City, State, Post..."
3,Contract manager drafts agreement using Contr...,Contract manager,drafts agreement,Contract,"Supplier ID, Partner ID, Contract Start Date,..."
4,Customer service agent provides order trackin...,Customer service agent,provides order tracking,Order,"Order ID, Tracking Number, Order Status"
...,...,...,...,...,...
1048570,Data Analyst formats data using Region detail...,Data Analyst,formats data,Region,"Region ID, Region Name, Country, State, City"
1048571,Data Analyst generates report using aggregate...,Data Analyst,generates report,aggregated data,"Total Sale Amount, Quantity, Region Name"
1048572,Data Analyst retrieves sales using Raw Data d...,Data Analyst,retrieves sales,Raw Data,"Raw Data ID, Timestamp, Sales Amount, Region ID"
1048573,Data Analyst reviews archived data using Regi...,Data Analyst,reviews archived data,Region,"Region ID, Region Name, Population"


In [ ]:
prompts = list()
for i in range(len(dataset.index)):

    prompt_for_finetune = f'''
    Provide Agent, CU,Entity, Attribute for the given Sentence in json format.

    INPUT:
    Sentence:
    {dataset.iloc[i].Sentence}


    Prediction:
    'Agent': {dataset.iloc[i].Agent}
    'CU': {dataset.iloc[i].CU}
    'Entity':{dataset.iloc[i].Entity}
    'Attribute':{dataset.iloc[i].Attribute}
    '''
    prompts.append(prompt_for_finetune)

dataset['prompt_for_finetune'] = prompts

In [ ]:
 prompt = f'''
    Provide Agent, CU,Entity, Attribute for the given Sentence in json format.
    INPUT:
    Sentence: System archives completed orders using Order details such as Order ID, Order Status, Delivery Date.

    '''

In [ ]:
from datasets import Dataset

datasets = Dataset.from_pandas(dataset[['prompt_for_finetune']])
datasets


Dataset({
    features: ['prompt_for_finetune'],
    num_rows: 1048575
})

In [ ]:
model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)

model.config.use_cache = False


config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/981M [00:00<?, ?B/s]

model-00002-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00003-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

model-00012-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00013-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00014-of-00014.safetensors:   0%|          | 0.00/847M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
# os.mkdir('/kaggle/working/results')

output_dir = "/kaggle/working/results"
per_device_train_batch_size = 2
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_steps = 50                 # Check points at which we would like to store the results
logging_steps = 5              # After this steps loss will be logged
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 50                  # Number of Epochs for which we will train the model.
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
max_seq_length = 1024

trainer = SFTTrainer(
    model=model,
    train_dataset=datasets,
    peft_config=peft_config,
    dataset_text_field="prompt_for_finetune",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments
)

Map:   0%|          | 0/1048575 [00:00<?, ? examples/s]

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,1.975100
10,2.299100
15,1.982400
20,1.526800
25,0.966900
30,0.812300
35,0.509900
40,0.595400
45,0.738800
50,0.455800


TrainOutput(global_step=50, training_loss=1.1862504386901855, metrics={'train_runtime': 135.1113, 'train_samples_per_second': 1.48, 'train_steps_per_second': 0.37, 'total_flos': 507575871897600.0, 'train_loss': 1.1862504386901855, 'epoch': 0.0})

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_to_save.save_pretrained("final_finetuned_model")

In [ ]:
# idx = 0
prompt_for_inference = f'''
    Provide Agent, CU,Entity, Attribute for the given Sentence in json format.

    INPUT:
    Sentence:
    {dataset.iloc[i].Sentence}


    Prediction:
    'Agent': {dataset.iloc[i].Agent}
    'CU': {dataset.iloc[i].CU}
    'Entity':{dataset.iloc[i].Entity}
    'Attribute':{dataset.iloc[i].Attribute}
    '''
prompt_for_inference

"\n    Provide Agent, CU,Entity, Attribute for the given Sentence in json format.\n\n    INPUT:\n    Sentence:\n     Data Analyst shares insights using report details such as Key Findings, Recommendations, Trends.\n\n\n    Prediction:\n    'Agent':  Data Analyst\n    'CU': shares insights\n    'Entity':report\n    'Attribute': Key Findings, Recommendations, Trends\n    "

In [ ]:
from peft import LoraConfig, get_peft_model

In [ ]:
lora_config = LoraConfig.from_pretrained('final_finetuned_model')
model = get_peft_model(model, lora_config)

In [ ]:

inputs = tokenizer(prompt_for_inference,return_tensors="pt")

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=50)

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<s> 
    Provide Agent, CU,Entity, Attribute for the given Sentence in json format.

    INPUT:
    Sentence:
     Data Analyst shares insights using report details such as Key Findings, Recommendations, Trends.


    Prediction:
    'Agent':  Data Analyst
    'CU': shares insights
    'Entity':report
    'Attribute': Key Findings, Recommendations, Trends
    
    OUTPUT:
    'Agent':  Data Analyst
    'CU': shares insights
    'Entity':report
    'Attribute': Key Findings, Recommendations, Trends


    """
